In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
path="/content/drive/MyDrive/Colab Notebooks/HW4&Final/FINAL"
os.chdir(path)
os.listdir(path)
import os
print(os.getcwd())

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/HW4&Final/FINAL


In [ ]:
!pip install accelerate
!pip install transformers


In [ ]:
!pip install datasets

In [ ]:
import pandas as pd

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
import torch

from transformers import BertForSequenceClassification, Trainer, TrainingArguments
!pip install datasets
from datasets import load_metric

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_metric
import numpy as np
from transformers import BertModel, GPT2Config, GPT2LMHeadModel
import torch.nn as nn


data_path = "AnonymizedClinicalAbbreviationsAndAcronymsDataSet.txt"

try:
    # 尝试使用 ISO-8859-1 编码读取数据
    data = pd.read_csv(data_path, sep='|', header=None, names=[
        "Short_form", "Long_form", "Abbreviation", "Start_pos", "End_pos", "Section_info", "Context"
    ], encoding='ISO-8859-1')
except UnicodeDecodeError:
    # 如果 ISO-8859-1 编码失败，则忽略错误的字符
    data = pd.read_csv(data_path, sep='|', header=None, names=[
        "Short_form", "Long_form", "Abbreviation", "Start_pos", "End_pos", "Section_info", "Context"
    ], encoding='ISO-8859-1', errors='ignore')

print(data.head())
print(data.info())

  Short_form Long_form Abbreviation  Start_pos  End_pos  \
0         AB  abortion          AB.      231.0    233.0   
1         AB  abortion          AB.      249.0    251.0   
2         AB  abortion           AB      223.0    224.0   
3         AB  abortion          AB.      194.0    196.0   
4         AB  abortion           AB      114.0    115.0   

                     Section_info  \
0                             NaN   
1                             NaN   
2                 PAST OB HISTORY   
3  HISTORY OF THE PRESENT ILLNESS   
4             PAST OB-GYN HISTORY   

                                             Context  
0  _%#NAME#%_ _%#NAME#%_ is a 29-year-old gravida...  
1  She is now bleeding quite heavily. Ultrasound ...  
2  ALLERGIES: Heparin and Imitrex. PAST OB HISTOR...  
3  She had a pelvic ultrasound at Park Nicollet o...  
4  On _%#MMDD2007#%_, normal anatomy with anterio...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37500 entries, 0 to 37499
Data columns (to

In [ ]:
# 假设 data 是 pandas DataFrame
labels = data['Long_form'].tolist()  # 从 DataFrame 中提取标签列
texts = data['Context'].tolist()  # 从 DataFrame 中提取文本列

# 创建从类别名称到整数索引的映射
label_list = sorted(set(labels))  # 创建一个包含所有唯一标签的列表
label_map = {label: idx for idx, label in enumerate(label_list)}

# 创建从整数索引到类别名称的逆映射
inverse_label_map = {idx: label for label, idx in label_map.items()}

# 打印标签映射以确保其正确性
print("Label Map:", label_map)
print("Inverse Label Map:", inverse_label_map)




print("Total labels:", len(label_map))
print("Total Inverse Label:", len(inverse_label_map))



# 将字符串标签转换为整数索引
labels_indexed = [label_map[label] for label in labels]

# 使用 train_test_split 分割数据集
from sklearn.model_selection import train_test_split
texts_train, texts_test, labels_train, labels_test = train_test_split(
    texts, labels_indexed, test_size=0.2, random_state=42)

Label Map: {'(PO Box) C4': 0, '(class) IA': 1, '(complement) component 3': 2, '(complement) component 4': 3, '(cycle) IB': 4, '(device) MP': 5, '(device) PD': 6, '(diltiazem) DC': 7, '(drug) AC': 8, '(drug) DC': 9, '(drug) DT': 10, '(drug) ES': 11, '(drug) IT': 12, '(drug) MP': 13, '(drug) MS': 14, '(drug) NP': 15, '(drug) OR': 16, '(drug) PAC': 17, '(drug) PD': 18, '(drug) PR': 19, '(drug) RT': 20, '(grade) IA': 21, '(grade) IB': 22, '(stage) C3': 23, '(stage) IA': 24, '(stage) IB': 25, '(status) IA': 26, '(status) IB': 27, '(type) IA': 28, '(type) IB': 29, 'American Society of Anesthesiologists': 30, 'American Society of Anesthesiologists:ASA': 31, 'BK (virus)': 32, 'C-reactive': 33, 'California': 34, "Children's Depression Inventory": 35, 'District of Columbia': 36, 'Fairview Southdale Hospital': 37, 'GENERAL ENGLISH': 38, 'Iowa': 39, 'Los Angeles': 40, 'Louisiana': 41, 'MISTAKE:EZ PAP': 42, 'MISTAKE:Oncotype DX': 43, 'MISTAKE:abduction': 44, 'Mall of America:MOA': 45, 'NAME': 46, '

In [ ]:


# 显示一些基本信息以确保数据正确处理
print(f"Number of training samples: {len(texts_train)}")
print(f"Number of test samples: {len(texts_test)}")
print("Example of processed data:")
print(texts_train[0], labels_train[0])


Number of training samples: 30000
Number of test samples: 7500
Example of processed data:
Will get his home medications and continue those. Will ask neurology to review his antiepileptic regimen and have spoken with Dr. _%#NAME#%_. We will have OT and PT evaluate the patient. Suspect that he may need a TCU versus home with additional services. LABORATORY DATA: That was performed here includes the following laboratory tests. 267


In [ ]:



class ClinicalTextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 加载预训练的Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification




# 在将数据传给tokenizer之前，确保所有文本都是字符串
texts_train = [str(text) for text in texts_train]
texts_test = [str(text) for text in texts_test]

# 现在尝试编码
train_encodings = tokenizer(texts_train, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(texts_test, truncation=True, padding=True, max_length=512)


# 编码数据
train_encodings = tokenizer(texts_train, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(texts_test, truncation=True, padding=True, max_length=512)

# 创建数据集
train_dataset = ClinicalTextDataset(train_encodings, labels_train)
test_dataset = ClinicalTextDataset(test_encodings, labels_test)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 加载预训练的BERT模型，添加一个顶层用于分类
#model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-v1.1", num_labels=len(label_map)).to(device)



class CustomBertForSequenceClassification(nn.Module):
    def __init__(self, pretrained_model_name, num_labels):
        super().__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.dropout = nn.Dropout(self.bert.config.hidden_dropout_prob)
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        pooled_output = self.dropout(outputs.pooler_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return {'loss': loss, 'logits': logits}

model = CustomBertForSequenceClassification('bert-base-uncased',  num_labels=len(label_map)).to(device)

#model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map)).to(device)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

metric = load_metric("accuracy")


def compute_metrics(preds, labels):
    pred_labels = np.argmax(preds, axis=1)
    accuracy = accuracy_score(labels, pred_labels)
    return accuracy

def evaluate(model, test_loader, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs['loss']
            logits = outputs['logits']
            total_loss += loss.item()
            all_preds.extend(logits.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    avg_loss = total_loss / len(test_loader)
    accuracy = compute_metrics(np.array(all_preds), np.array(all_labels))
    return avg_loss, accuracy

def train(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs['loss']
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    return avg_loss

def train_model(model, train_loader, valid_loader, device, num_epochs=3):
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for step, batch in enumerate(train_loader):
            input_ids, attention_mask, labels = (
                batch['input_ids'].to(device),
                batch['attention_mask'].to(device),
                batch['labels'].to(device)
            )
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs['loss']

            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

            if step % 100 == 0:
                print(f"Epoch {epoch + 1}, Step {step}, Loss: {loss.item()}")

        print(f"Epoch {epoch + 1}, Average Training Loss: {total_train_loss / len(train_loader)}")

        val_loss, val_accuracy = evaluate(model, valid_loader, device)
        print(f"Epoch {epoch + 1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
def zero_shot_evaluation(model, valid_loader, device):
    val_loss, val_accuracy = evaluate(model, valid_loader, device)
    print(f"Zero-Shot Evaluation - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")



<ipython-input-8-0bd34b37a9e8>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# Zero-Shot 评估
zero_shot_evaluation(model, test_loader, device)


Zero-Shot Evaluation - Loss: 5.9616, Accuracy: 0.0103


In [ ]:
# 开始训练
train_model(model, train_loader, test_loader, device)

Epoch 1, Step 0, Loss: 6.064361095428467
Epoch 1, Step 100, Loss: 4.793985366821289
Epoch 1, Step 200, Loss: 2.729743242263794
Epoch 1, Step 300, Loss: 1.4613012075424194
Epoch 1, Step 400, Loss: 1.0998822450637817
Epoch 1, Average Training Loss: 2.7573844200766673
Epoch 1, Validation Loss: 0.8496, Validation Accuracy: 0.8433
Epoch 2, Step 0, Loss: 0.6738908886909485
Epoch 2, Step 100, Loss: 0.7207075357437134
Epoch 2, Step 200, Loss: 0.82240891456604
Epoch 2, Step 300, Loss: 0.4646555781364441
Epoch 2, Step 400, Loss: 0.49566400051116943
Epoch 2, Average Training Loss: 0.5950231282060334
Epoch 2, Validation Loss: 0.4206, Validation Accuracy: 0.9189
Epoch 3, Step 0, Loss: 0.3884165585041046
